In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="7"
import shfl
import torch
import copy
import numpy as np

from sklearn.preprocessing import LabelBinarizer

from shfl.private import UnprotectedAccess
from CIT.model import CITModel
from utils import get_federated_data_csv
from ClassifierModel import ClassifierModel

/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/frame

In [6]:
#"../data/COVIDGR1.0/centralized/cropped"
#partition_iid_3nodes_1.csv
args = {"data_path":"../data/COVIDGR1.0reducido-cropped", 
        "csv_path": "../partitions/partition1reducido.csv",
        "output_path": "../weights",
        "input_path": "",
        "model_name":"transferlearning.model", 
        "label_bin": "lb.pickle", 
        "batch_size": 1,
        "federated_rounds": 1,
        "epochs_per_FL_round": 1,
        "num_nodes": 3,
        "size_averaging": 1,
        "random_rotation": 0,
        "random_shift": 0, 
        "random_zoom": 0,
        "horizontal_flip": False,        
        "finetune": True,
        "train_network": True}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
def cit_builder():    
    return CITModel(['N', 'P'], classifier_name = "resnet18", lambda_value = 0.00075, batch_size=args["batch_size"], epochs=args["epochs_per_FL_round"], device=device)

def classifier_builder():
    return ClassifierModel(batch_size=args["batch_size"], epochs=args["epochs_per_FL_round"], finetune = args["finetune"])

def get_transformed_data(federated_data, cit_federated_government, lb1, lb2):
    t_federated_data = copy.deepcopy(federated_data)

    for i in range(federated_data.num_nodes()):
        data_node = federated_data[i]
        t_data_node = t_federated_data[i]
        data = data_node.query()._data
        labels = data_node.query()._label
        t_data, t_labels = cit_federated_government.global_model.transform_data(data, labels, lb1, lb2)
        t_data_node.query()._data = t_data
        t_data_node.query()._label = t_labels

    t_test_data, t_test_label = cit_federated_government.global_model.transform_data(test_data, test_label, lb1, lb2)

    return t_federated_data, t_test_data, t_test_label

In [8]:
a = ['N', 'P']
b = ['NTN', 'NTP', 'PTP', 'PTN']

lb1 = LabelBinarizer()
lb2 = LabelBinarizer()

lb1.fit(a)
lb2.fit(b)

LabelBinarizer()

In [9]:
print("[INFO] Fetching federated data...")
federated_data, train_data, train_label, test_data, test_label, train_files, test_files = get_federated_data_csv(args["data_path"], args["csv_path"], lb1)
federated_data.configure_data_access(UnprotectedAccess())
print("[INFO] done")

[INFO] Fetching federated data...
[INFO] done


In [44]:
aggregator = shfl.federated_aggregator.FedAvgAggregator()
cit_federated_government = shfl.federated_government.FederatedGovernment(cit_builder, federated_data, aggregator)
cit_federated_government.run_rounds(args["federated_rounds"], test_data, test_label)

0it [00:00, ?it/s]

[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
Accuracy round 0
[INFO] weights = [1. 1.]


0it [00:00, ?it/s]
[Validating]: Acc_D: 0.5000: 100%|██████████| 8/8 [00:00<00:00,  9.31it/s]
0it [00:00, ?it/s]

Valid Loss = 4.355374708771706
Valid Acc = 0.5
[INFO] weights = [1. 1.]


0it [00:00, ?it/s]
[Validating]: Acc_D: 0.5000: 100%|██████████| 8/8 [00:00<00:00,  9.19it/s]


Valid Loss = 4.094979844987392
Valid Acc = 0.5
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x7f6204401a20>: [3.8566150665283203, 0.5]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x7f6204401630>: [3.8566150665283203, 0.5]
Global model test performance : [0.7781913876533508, 0.5]





In [45]:
t_federated_data, t_test_data, t_test_label = get_transformed_data(federated_data, cit_federated_government, lb1, lb2)

In [46]:
aggregator = shfl.federated_aggregator.FedAvgAggregator()
classifier_federated_government = shfl.federated_government.FederatedGovernment(classifier_builder, t_federated_data, aggregator)
classifier_federated_government.run_rounds(args["federated_rounds"], t_test_data, t_test_label)

#print("[INFO] saving model ...")
#federated_government.global_model._model.save( os.path.join(args["output_path"], args["model_name"]) )
print("[INFO] done")

Accuracy round 0
14/14 [==============================] - 18s 1s/step - loss: 1.7158 - categorical_accuracy: 0.0000e+00 - val_loss: 1.3784 - val_categorical_accuracy: 0.0000e+00
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x7f6143b028d0>: [1.4062868356704712, 0.25]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x7f62045d2710>: [1.427771806716919, 0.375]
Global model test performance : [1.4048292636871338, 0.125]



[INFO] done


In [47]:
dict_labels = { 'PTP' : np.argmax(lb2.transform(['PTP'])[0]) , 'PTN' : np.argmax(lb2.transform(['PTN'])[0]) , 
                'NTP' : np.argmax(lb2.transform(['NTP'])[0]) , 'NTN' : np.argmax(lb2.transform(['NTN'])[0])
              } 
G_dict = cit_federated_government.global_model._G_dict

for key, _ in G_dict.items():
    G_dict[key].to("cpu")

classifier_federated_government.global_model.get_classification_report(test_files, dict_labels, G_dict, "qaa")

Accuracy 4 clases: 0.25
Numero de veces no concuerda: 4
{'N': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2}, 'P': {'precision': 0.5, 'recall': 1.0, 'f1-score': 0.6666666666666666, 'support': 2}, 'accuracy': 0.5, 'macro avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 4}, 'weighted avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 4}}
0.5


/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
